---
title: Running TSEB/3SEB with Satellite Imagery
subject: Tutorial
subtitle: Notebook to pre-process and run TSEB/3SEB using Copernicus-based satellite inputs
short_title: Copernicus_3SEB
authors:
  - name: Vicente Burchard-Levine
    affiliation:
      - SpecLab-CSIC
    orcid: 0000-0003-0222-8706
    email: vburchard@ica.csic.es
  - name: Radoslaw Guzinski
    affiliations:
      - DHI-GRAS
    orcid: 0000-0003-0044-6806
  - name: Héctor Nieto
    affiliations:
      - ICA-CSIC
    orcid: 0000-0003-4250-6424
    email: hector.nieto@ica.csic.es
license: CC-BY-SA-4.0
keywords: TSEB, 3SEB, Copernicus, Satellite
---

# Summary 

Interactive jupyter notebook showing the implementation of 3SEB with satellite imagery from the Copernicus program, namely from Sentinel-2 and Sentinel-3. 

This notebook will go through the pre-processing of satellite-based imagery, preparing all necesary inputs and runnign TSEB/3SEB. This includes:

- Pre-processing and preparing satellite imagery
- Sharpening Sentinel-3 LST
- Estimating biopysical variables and ancillary parameters
- Running TSEB and 3SEB

# Instructions
Read carefully all the text and follow the instructions.

:::{hint} 

Once each section is read, run the jupyter code cell underneath (marked as `[]`) by clicking the icon `Run`, or pressing the keys SHIFT+ENTER of your keyboard.


:::

To start, please run the following cell to import all the packages required for this notebook. Once you run the cell below, an acknowledgement message, stating all libraries were correctly imported, should be printed on screen.

:::{warning}

Most of the functions implemented in this notebook are credited to the **EOMAJI-OpenEO-Toolbox** (found here: *https://github.com/DHI/EOMAJI-OpenEO-toolbox/*). 

This project is under active development. Features may change and bugs may exist.
:::


# Import Libraries

In [ ]:
%matplotlib widget
import os 
import openeo
import numpy as np
from osgeo import gdal
from pathlib import Path
import pandas as pd
import geopandas as gpd
import rasterio
import xarray
from pyTSEB import TSEB
from pyTSEB import meteo_utils as met
from pyTSEB import net_radiation as rad
from pyTSEB import resistances as res
from py3seb import  py3seb 
import matplotlib.pyplot as plt
from functions import gdal_utils as gu
from functions.eomaji.utils import draw_utils, date_selector
from functions.eomaji.utils.general_utils import dump_area_date_info, read_area_date_info
from functions.eomaji.workflows import prepare_data_cubes
from functions.eomaji.workflows.decision_tree_sharpener import run_decision_tree_sharpener
from functions.eomaji.workflows.sentinel2_preprocessing import split_datasets_to_tiffs
from functions.eomaji.workflows.meteo_preprocessing import get_meteo_data
from functions.eomaji.utils.raster_utils import resample_to_s2

print('libraries imported correctly!')

# General workflow

The general workflow to run TSEB/3SEB using Sentinel imagery from the Copernicus program is depicted below. 

The main processing steps are:
1. Acquring and pre-processing Sentinel-2 and Sentinel-3 imagery from the [**CDSE**](https://dataspace.copernicus.eu/)
2. Performing TIR sharpening on Sentinel-3 LST (1km ->20m) using [**pyDMS**](https://github.com/radosuav/pyDMS)
3. Estimating biophysical variables (e.g. LAI) from RTM inversion using [**pypro4sail**](https://github.com/hectornieto/pypro4sail)
4. Retrieving ancillary vegetation parameters per plant functional type based on global land cover map.
5. Acquiring and pre-processing ERA-5 reanalysis meteo data using [**meteo_utils**](https://github.com/hectornieto/meteo_utils)
6. Running TSEB and 3SEB using [**pyTSEB**](https://github.com/hectornieto/pyTSEB) and [**py3SEB**](https://github.com/VicenteBurchard/py3SEB), respectively 

:::{figure} ./input/figures/S2_processing_flowchart.png
:alt: S2
:name: S2-flowchart
TSEB input processing using Copernicus datasets (figure credit: Héctor Nieto)
:::

:::{note}
While the figure above depicts the use of Landsat imagery to further correct the sharpened LST product, we won´t be implementing this in this notebook as we will solely focus on the use of Copernicus products from the CDSE. 

The use of Landsat to better capture the high resolution LST variability during the sharpening process is based on the results from [**Guzinski et al. 2023**](https://doi.org/10.1016/j.jag.2023.103587).
:::

:::{note}
Futhermore, the global Land Cover map used in this notebook is ESA's 10m [**WorldCover2021**](https://worldcover2021.esa.int/) instead of the 100m [**Global Dynamic Land Cover**](https://land.copernicus.eu/en/products/global-dynamic-land-cover) map fom Copernicus Land Monitoring Service. 
:::



# Requirements

:::{important}

In order to execute this notebook, you will need to register in the [**Copernicus Data Space Ecosystem (CSDE)**](https://dataspace.copernicus.eu/) to process Sentinel-2 and Sentinel-3 imagery.

Along with this, to acquire ERA-5 weather data you should first register to the Copernicus' [**Climate Data Store**](https://cds.climate.copernicus.eu/user/register)
and [Atmospheric Data Store](https://ads.atmosphere.copernicus.eu/user/register) systems.

Once registered, follow the steps in the [**CDS User Guide**](https://cds.climate.copernicus.eu/how-to-api) where you will get a personal URL and KEY for both CDS and ADS.

You will then need to create a ```.adsapirc``` and a ```.cdsapirc``` file with the API key like this:

``` bash
.adsapirc
url: https://ads.atmosphere.copernicus.eu/api
key: <api_key>
```
and
```` bash
.cdsapirc
url: https://cds.climate.copernicus.eu/api
key: <api_key>
````

Examples files are provided in./.adsapirc_example and ./cdsapirc_example.

:::

# Data preparation 

This notebook will make use of Sentinel data freely available from the [**Copernicus Data Space Ecosystem (CDSE)**](https://dataspace.copernicus.eu/) platform. 

The first step is to generate data cubes of the datasets needed over you area of interest. 

:::{important}
To access the data and execute the notebook, you will need to register a free account here: *https://dataspace.copernicus.eu/*
:::

## Select area of interest 

You have the option of drawing a polygon on the map, for the area you want to process or you can specifying the bounding box (bbox) with the format **[minx, miny, maxx, maxy]** i.e. **[min_lon, min_lat, max_lon, max_lat]**

:::{note}

We have pre-selected a bounding box around the WES experimental Almond Orchard with bbox = [-121.35,37.45, -121.10, 37.65]. Change this or use the map tool to draw a polygon to process a different area. However, the notebook is designed to be used over the WES Almond Orchard.
:::

:::{warning}

We suggest to start with small region of interest to limit processing time. For large areas the download and aggregation of data in OpenEO may take some time and fail.

:::



In [ ]:
map, bboxs = draw_utils.draw_aoi()
map

## Select a date from the available Sentinel-3 imagery

Here, we will select satellite imagery over the same data and area as the UAV overpass from the WES almond orchard as from the previous notebook (./403-UAV-3SEB.ipynb).

As mentioned before, the bbox is set over the WES almond orchard area of interest but if you want to use the area that you drew on the map, you can uncomment 
``` bash
#bbox = bboxs[-1]  # Uncomment this if you want to use the polygon drawn above

```


In [ ]:
# Define search parameters
start_date = "2024-04-16"
end_date = "2024-04-16"
bbox = [-121.35,37.45, -121.10, 37.65] # please insert a bbox here in the form of [minx, miny, maxx, maxy]
#bbox = bboxs[-1] # Uncomment this if you want to use the polygon drawn above

max_cloud_cover = 10  # Filter out high-cloud-coverage scenes

# Search for available Sentinel-3 imagery
date_selection = date_selector.get_available_dates(
    start_date=start_date,
    end_date=end_date,
    bbox=bbox,
    max_cloud_cover=max_cloud_cover
)

## Connect to OpenEO Backend

In [ ]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

## Pre-process Sentinel 2 and Sentinel 3 data for the specified AOI and date

We will create a folder to save the processed imagery in **./dataset/sentinel_imagery**


Specify the path to store the processed satellite imagery

In [ ]:
# specify directory to save satellite imagery
data_dir = "./dataset/sentinel_imagery"
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

date = date_selection.value
#date, bbox = read_area_date_info(
#    dir=data_dir
#) # <-- USE THIS IF YOU WANT TO REUSE BBOX AND DATE FROM FILE

Now we will begin the pre-proceessing of sentinel-2 and sentinel-3 imagery and generate data cubes

:::{warning}

As mentioned previously, this section might take some time, especially for larger areas. For the pre-selected AOI over the WES almond orchard, it could take **over 15-20 minutes**. 

You can go to **https://openeo.dataspace.copernicus.eu/** and sign in to your account and follow the status of your jobs and see any errors.

:::

In [ ]:
s2_path, s3_path, vza_path, worldcover_path, dem_s2_path, dem_s3_path, sentinel3_acq_time, = prepare_data_cubes.prepare_data_cubes(
    connection=connection,
    bbox=bbox,
    date=date,
    sentinel2_search_range = 3,
    out_dir = data_dir,
)

### Store AOI and data for subsequent use

In [ ]:
dump_area_date_info(
    date = date, 
    bbox = bbox, 
    out_dir = data_dir
)

## Visualize sentinel-2 and Sentinel-3 data cubes

Here, we can inspect the sentinel imagery.

Sentinel-2 band information:

:::{figure} ./input/figures/S2_bands_info.jpg
:alt: S2
:name: S2-bands
Sentinel-2 MSI bands information (figure taken from [Pasqualotto et al. 2019](https://ieeexplore.ieee.org/document/8909218))
:::



In [ ]:
# open sentinel-2 data
s2_cube =  xarray.open_dataset(s2_path)
band_names = ["B02", "B03", "B04", "B05", "B07", "B08", "B8A", "B11", "B12"]  # These are the Sentinel 2 bands to use for the sharpening
s2_array = s2_cube[band_names].to_array(dim="band").rio.write_crs(rasterio.crs.CRS.from_string(s2_cube.crs.spatial_ref).to_string())

# open sentinel-3 data
s3_cube = xarray.open_dataset(s3_path)
lst_array = s3_cube.LST.rio.write_crs(rasterio.crs.CRS.from_string(s3_cube.crs.spatial_ref).to_string())

# get extent [minx, maxx, miny, maxy]
te = [float(s2_cube['x'].min()), float(s2_cube['x'].max()), float(s2_cube['y'].min()), float(s2_cube['y'].max())]

# Read specific bands (5=NIR, 2=Red,1=green and 0=blue)
nir_band = s2_array[5,:,:]
red_band = s2_array[2,:,:]
green_band = s2_array[1,:,:]
blue_band = s2_array[0,:,:]

# calc NDVI
ndvi_ar = (nir_band - red_band)/(nir_band + red_band)

# normalize to visualize RGB
red_norm = (red_band - np.nanmin(red_band)) / (np.nanmax(red_band) - np.nanmin(red_band))  # scale 0–1
green_norm = (green_band - np.nanmin(green_band)) / (np.nanmax(green_band) - np.nanmin(green_band))  # scale 0–1
blue_norm = (blue_band - np.nanmin(green_band)) / (np.nanmax(blue_band) - np.nanmin(blue_band))  # scale 0–1

rgb_stack =  np.dstack((red_norm, green_norm, blue_norm))

# plot imagery
fig, axes = plt.subplots(2,3, figsize = (12,8), constrained_layout=True)
ax = axes[0,0]
ax.imshow(rgb_stack*5.5, extent = te)
ax.set_title('S2-True Color', fontsize=12)

ax = axes[0,1]
im1 = ax.imshow(ndvi_ar, extent = te, cmap='YlGn', vmin=0.3, vmax=0.7)
ax.set_title('S2-NDVI', fontsize=12)
cb = plt.colorbar(im1, ax=ax, shrink=0.65)
cb.set_label(' (-)', fontsize=14)

ax = axes[0,2]
im2 = ax.imshow(lst_array.values[0,:,:], extent = te, cmap='coolwarm', vmin=290, vmax=310)
ax.set_title('S3-LST', fontsize=12)
cb = plt.colorbar(im2, ax=ax, shrink=0.65)
cb.set_label('(K)', fontsize=14)

# zoom to WES almonds
ax = axes[1,0]
ax.imshow(rgb_stack*5.5, extent = te)
ax.set_title('Zoom to Almond orchard', fontsize=12)
ax.set_xlim(654900, 655850)
ax.set_ylim(4156600, 4157500)

# zoom to WES almonds
ax = axes[1,1]
im1 = ax.imshow(ndvi_ar, extent = te, cmap='YlGn', vmin=0.3, vmax=0.7)
ax.set_title('Zoom to Almond orchard', fontsize=12)
ax.set_xlim(654900, 655850)
ax.set_ylim(4156600, 4157500)

# zoom to WES almonds
ax = axes[1,2]
im2 = ax.imshow(lst_array.values[0,:,:], extent = te, cmap='coolwarm', vmin=290, vmax=310)
ax.set_title('Zoom to Almond orchard', fontsize=12)
ax.set_xlim(654900, 655850)
ax.set_ylim(4156600, 4157500)
plt.show()


:::{note}
As you can see, we do not have any plot-level information with the LST data from Sentinel-3 because it is too coarse.

For this reason, it is important to perform an LST sharpening to obtain high resolution LST at the sub-plot level
:::

# Sentinel-3 LST Sharpening

Sentinel-3 SLSTR sensor provides thermal infrared (TIR) images at 1km spatial resolution but we can use the information from Sentinel-2 images (with 20m pixel size) to sharpnen the TIR images to 20m to obtain information at the subplot scale, especially important for agricultural applications. 

In this case, we will use the Data Mining Sharpener (DMS) which was first proposed by Gao et al. (2012) (for more information see paper [**here**](https://doi.org/10.3390/rs4113287)) which assumes a relationship between optical shortwave bands and LST. We will use the pyDMS code (available here: https://github.com/radosuav/pyDMS) to perform this sharpening.

## Open S2 and S3 data cubes

In [ ]:
# open sentinel-2 data
s2_cube =  xarray.open_dataset(s2_path)
band_names = ["B02", "B03", "B04", "B05", "B07", "B08", "B8A", "B11", "B12"]  # These are the Sentinel 2 bands to use for the sharpening
s2_array = s2_cube[band_names].to_array(dim="band").rio.write_crs(rasterio.crs.CRS.from_string(s2_cube.crs.spatial_ref).to_string())

# open sentinel-3 data
s3_cube = xarray.open_dataset(s3_path)
lst_array = s3_cube.LST.rio.write_crs(rasterio.crs.CRS.from_string(s3_cube.crs.spatial_ref).to_string())
mask_array = ((s3_cube.confidence_in < 16384).astype(float).rio.write_crs(rasterio.crs.CRS.from_string(s3_cube.crs.spatial_ref).to_string()))
print('Done!')

## Run the Data Mining Sharpener
Here we run the data mining sharpening algorithm which is based on [**pyDMS**](https://github.com/radosuav/pyDMS)

In [ ]:
sharpened_data = run_decision_tree_sharpener(
    high_res_dataarray=s2_array,
    low_res_dataarray=lst_array,
    low_res_mask=mask_array,
    mask_values=[1],
    cv_homogeneity_threshold=0,
    moving_window_size=30,
    disaggregating_temperature=True,
    n_jobs=3,
    n_estimators=30,
    max_samples=0.8,
    max_features=0.8,
)

# specify output path for sharpened LST
lst_outdir = Path(s3_path).parent/"sharpened_LST.tif"

print(f'Saving sharpened LST to {lst_outdir}')
sharpened_data.band_data.rio.to_raster(lst_outdir)

print('Done!')

### Visualize LST sharpening
Here we plot the sharpened images but you can also inspect the images using QGIS

In [ ]:
# plot imagery
fig, axes = plt.subplots(2,3, figsize = (12,8), constrained_layout=True)

ax = axes[0,0]
ax.imshow(rgb_stack*5.5, extent = te)
ax.set_title('S2-True Color', fontsize=12)

ax = axes[0,1]
im2 = ax.imshow(lst_array.values[0,:,:], extent = te, cmap='coolwarm', vmin=290, vmax=310)
ax.set_title('S3-LST (1km)', fontsize=12)
#cb = plt.colorbar(im2, ax=ax, shrink=0.65)
#cb.set_label('(K)', fontsize=14)

ax = axes[0,2]
im1 = ax.imshow(sharpened_data['band_data'].values[0,:,:], extent = te, cmap='coolwarm',vmin=290, vmax=310)
ax.set_title('S3-LST-Sharpened (20m)', fontsize=12)
cb = plt.colorbar(im1, ax=ax, shrink=0.7)
cb.set_label(' (K)', fontsize=12)

# zoom to WES almonds
ax = axes[1,0]
ax.imshow(rgb_stack*5.5, extent = te)
ax.set_title('Zoom to Almond orchard', fontsize=12)
ax.set_xlim(654900, 655850)
ax.set_ylim(4156600, 4157500)

# zoom to WES almonds
ax = axes[1,1]
im2 = ax.imshow(lst_array.values[0,:,:], extent = te, cmap='coolwarm', vmin=290, vmax=310)
ax.set_title('Zoom to Almond orchard', fontsize=12)
ax.set_xlim(654900, 655850)
ax.set_ylim(4156600, 4157500)

# zoom to WES almonds
ax = axes[1,2]
im2 = ax.imshow(sharpened_data['band_data'].values[0,:,:], extent = te, cmap='coolwarm', vmin=290, vmax=310)
ax.set_title('Zoom to Almond orchard', fontsize=12)
ax.set_xlim(654900, 655850)
ax.set_ylim(4156600, 4157500)
plt.show()


# Meteo and biophysical processing

Now we need to process the other TSEB/3SEB inputs including the biophysical variables and meterological forcings

Re-Check if all necessary imagery were pre-processed correctly 


In [ ]:
s2_path, s3_path, vza_path, worldcover_path, dem_s2_path, dem_s3_path, sentinel3_acq_time = prepare_data_cubes.prepare_data_cubes(
    connection=connection,
    bbox=bbox,
    date=date,
    sentinel2_search_range = 3,
    out_dir = data_dir,
)

# correct sentinel-3 acquisition time to UTC (seems that the function does not produce correct UTC time)
date_ts = pd.to_datetime(s3_cube.t.values[0])
sentinel3_acq_time = float(date_ts.hour)
print('Done!')

## Estimation of biophysical variables

The **prepare_data_cubes** function from the [EOMAJI-OpenEO-Toolbox](https://github.com/DHI/EOMAJI-OpenEO-toolbox/blob/main/eomaji/workflows/prepare_data_cubes.py#L58)  internally estimates the necessary biophysical variables (i.e. LAI, Fg (green fraction), Fc (veg. fractional cover)) using the [**BIOPAR**](https://marketplace-portal.dataspace.copernicus.eu/catalogue/app-details/21#iss=https%3A%2F%2Fidentity.dataspace.copernicus.eu%2Fauth%2Frealms%2FCDSE) tool from the CDSE. 

This method essentially implements a hybrid radiative transfer modeling (RTM) method where a machine learning (ML) algorithm (e.g. Artificial Neural Network-ANN) to estimate biophysical variables is trained using synthetic top-of-canopy reflectances from the RTM (i.e. PROSAIL) and then applied to the Sentinel-2 bands to retrieve the different variables.

:::{figure} ./input/figures/Biophysical_processor_figure.png
:alt: S2-BIOPAR
:name: S2-BIOPAR
Flow chart showing how the BIOPAR products are generated operationally (figure taken from [S2 Toolbox ATDB](https://step.esa.int/docs/extra/ATBD_S2ToolBox_V2.1.pdf))
:::


You can find the documentation of the biophysical processor [**here**](https://step.esa.int/docs/extra/ATBD_S2ToolBox_V2.1.pdf) and an adapted and more efficient version of this method can be found in the [**pypro4sail**](https://github.com/hectornieto/pypro4sail) python package. 

:::{note}

The notebook **./502-Biophysical_Traits_RTM.ipynb** goes into more details on how to implement these types of methods for estimating biophysical traits, offerring a step-by-step guide.

:::

Other vegetation parameters, such as canopy height (Hc), which are not estimated through the RTM inversion, need to be estimated using other methods. In this case, these are estimated using a look-up-table over a global land cover map, assigning vegetation parameters based on different plant functional types. In this case of Hc, values are then scaled based on LAI for crops or herbaceous vegetation which have strong phenological dynamics. 

In other applications, ancillary non-copernicus datasets can be ingested, such as [**GEDI's**](https://gedi.umd.edu/data/products/) canopy height product, which can serve to characterize Hc in forested ecosystems.

You can find more information in Guzinski et al. ([2020](https://doi.org/10.3390/rs12091433), [2021](https://doi.org/10.1109/JSTARS.2021.3122573), [2023](https://doi.org/10.1016/j.jag.2023.103587)).

In the [**EOMAJI-OpenEO-Toolbox**](https://github.com/DHI/EOMAJI-OpenEO-toolbox/). , this procedure is done internally using the **split_datasets_to_tiffs** function after the sentinel-3 pre-processing.

In [ ]:
tif_path = split_datasets_to_tiffs(s2_path = s2_path, s3_path = s3_path, worldcover_path = worldcover_path, date = date)

### Visualize biophysical inputs 


In [ ]:
variables = {'LAI':{'cmap':'YlGn', 'range':[0,5]},
             'FCOVER':{'cmap':'Greens', 'range':[0,1]},
             'H_C':{'cmap':'cividis', 'range':[0,10]}
            }

cmaps = ['PiYG', 'Greens', 'cividis']

# make a 2x3 
fig, axes = plt.subplots(2,3, figsize = (12,8), constrained_layout=True)
i = 0
for var in variables.keys():
    filename = tif_path/f'{str(date)[:4]}{str(date)[5:7]}{str(date)[-2:]}_{var}.tif'
    fid = gdal.Open(str(filename))
    ar = fid.GetRasterBand(1).ReadAsArray()
    
    ax = axes[0,i]
    ax.imshow(ar, extent = te, cmap=variables[var]['cmap'], vmin=variables[var]['range'][0], vmax=variables[var]['range'][1])
    ax.set_title(f'{var}', fontsize=12)

    # zoom to WES almonds
    ax = axes[1,i]
    ax.imshow(ar, extent = te, cmap=variables[var]['cmap'], vmin=variables[var]['range'][0], vmax=variables[var]['range'][1])
    ax.set_title('Zoom to Almond orchard', fontsize=12)
    ax.set_xlim(654900, 655850)
    ax.set_ylim(4156600, 4157500)
    i = i + 1
    
plt.show()


:::{tip} Question
In the WES almond orchard, do the biophysical values seem reasonable? What about the spatial pattern of Hc? Have a look at the land use map (*./dataset/sentinel_imagery/20240416_ad419755/WordlCover2021.tif*) and see how the orchard is classified? (see WorldView documentation [**here**](https://worldcover2021.esa.int/data/docs/WorldCover_PUM_V2.0.pdf))

It is often challenging to apply global methods over tree orchards which have complex characteristics and tend to be misclassified or simply classified as CROP even though they have quite different structural characteristics, especially those with co-ocurring cover crops.
::: 

:::{tip} Question
How do you think this will affect the TSEB modeling and flux outputs?
:::

## Get ERA-5 Meteo data 

Fetch data from Atmosphere Data Store and Climate Data Store and calculate meteorological forcings.

The notebook depends on [**Climate Data Store**](https://cds.climate.copernicus.eu/) and the [**Atmosphere Data Store**](https://cds.climate.copernicus.eu/). To access data from these two sources, you need to create a profile in the [**Climate Data Store**](https://cds.climate.copernicus.eu/) and obtain an API key as described in the documentation:
* [**CDS User Guide**](https://cds.climate.copernicus.eu/how-to-api)


:::{important}

To run the next functions, you need to create a ```.adsapirc``` and a ```.cdsapirc``` file with the API key like this:

``` bash
.adsapirc
url: https://ads.atmosphere.copernicus.eu/api
key: <api_key>
```
and
```` bash
.cdsapirc
url: https://cds.climate.copernicus.eu/api
key: <api_key>
````
:::


In [ ]:
# Specify path to .cdsapirc and .adsapirc files with your URL and KEY
cds_file = "./.cdsapirc"
ads_file = "./.adsapirc"

meteo_output_path = get_meteo_data(
    date = str(date),
    bbox = bbox,
    dem_path = dem_s3_path,
    acq_time = sentinel3_acq_time,
    data_dir=dem_s3_path.parent,
    cds_credentials_file=cds_file,
    ads_credentials_file=ads_file,
)
print('Done!')

# Running TSEB

Now, we have all necessary inputs to run TSEB.

### Open satellite imagery as arrays

Here, we will create input dictionary (i.e. input_dict) which will store all the inputs necessary to run TSEB and 3SEB

In [ ]:
# dictioanry to store inputs arrays
input_dict = {}

# input directory
input_dir = tif_path

# meteo dir
meteo_dir = input_dir / 'meteo_data'

# store inputs as in input dict
input_vars = ['LST', 'LAI', 'F_G', 'FCOVER', 'H_C', 'W_C', 'VZA', 'SZA', 'RHO_NIR_C', 'RHO_VIS_C', 'TAU_NIR_C', 'TAU_VIS_C']

for var in input_vars:
    if var == 'LST':
        filename = input_dir / 'sharpened_LST.tif'
    else:
        filename = input_dir/f'{str(date)[:4]}{str(date)[5:7]}{str(date)[-2:]}_{var}.tif'

    # get raster info 
    if var == 'LAI':
        proj, gt, x_size, y_size, extent, center_geo, _ = gu.raster_info(str(filename))

    fid = gdal.Open(str(filename))
    ar = fid.GetRasterBand(1).ReadAsArray()
    # store within input dictionary
    input_dict[var] = ar 
    

meteo_var = ["EA", "p", "u", "S_dn_24", "S_dn", "T_A1", 'LW-IN', 'ETR']
# use LAI as a template
template_file = Path(tif_path)/f"{str(date).replace('-', '')}_LAI.tif"

for var in meteo_var:
    filename = list(meteo_dir.glob(f'*{var}.tif'))[0]
    # resample to sentinel-2 resolution
    fid = gu.resample_with_gdalwarp(filename, template_file, "bilinear")
    #fid = gdal.Open(str(filename))
    ar = fid.GetRasterBand(1).ReadAsArray()
    input_dict[var] = ar 

# parameterize constant variables
constant_params = {'RHO_NIR_S':0.07, # reflectance of soil in NIR
                  'RHO_VIS_S':0.28, # reflectance of soil in VIS
                  'E_S':0.95, # soil emissivity
                  'E_V':0.99, # vegetation emissivity
                  'Z0_SOIL':0.01, # soil roughness
                  'KN_C':0.0038, # Kondo & Ishida (1997) coefficient for rough surfaces
                  'KN_B':0.0120, # Kustas and Norman (1999) after Sauer and Norman (1995)
                  'ALPHA_PT':1.26, # alpha parameter in Priestley-Taylor Initialization
                  'X_LAD':1., # Chi parameter for leaf angle distribution
                  'Fc':1, # fractional cover of vegetation
                  'Fg':1, # fraction of vegetation that is photosynthetically active
                  'LEAF_WIDTH':0.05 # effective leaf width
                      
                  }

for key in constant_params.keys():
    ar = np.full_like(input_dict['LAI'], constant_params[key])
    input_dict[key] = ar 


print(f'Inputs stored in input_dict: \n{list(input_dict.keys())}')

### Net Radiation and clumping

In [ ]:
# calculate diffuse/direct ratio
difvis, difnir, fvis, fnir = TSEB.rad.calc_difuse_ratio(input_dict['S_dn'], input_dict['SZA'], press=input_dict['p'])
skyl = fvis * difvis + fnir * difnir
Sdn_dir = (1. - skyl) * input_dict['S_dn']
Sdn_dif = skyl * input_dict['S_dn']

# incoming long wave radiation
emisAtm = rad.calc_emiss_atm(input_dict['EA'], input_dict['T_A1'])
Lsky = emisAtm * met.calc_stephan_boltzmann(input_dict['T_A1'])

# We need to compute SAA
doy = date_ts.dayofyear
dec_time = date_ts.hour + date_ts.minute / 60.
sza, saa = met.calc_sun_angles(center_geo[1], center_geo[0], 0,doy, dec_time)


# to take into account row strucure on vegetation clumping
row_direction = 90
psi = row_direction - saa


Omega0 = TSEB.CI.calc_omega0_Kustas(input_dict['LAI'], input_dict['FCOVER'], x_LAD=input_dict['X_LAD'], isLAIeff=True)
Omega = TSEB.CI.calc_omega_rows(input_dict['LAI'], input_dict['FCOVER'], theta=input_dict['SZA'],
                                psi=psi, w_c=input_dict['W_C'],
                                x_lad=input_dict['X_LAD'])

F = input_dict['LAI']/input_dict['FCOVER']
# effective LAI (tree crop)
input_dict['LAI_EFF'] =  F * Omega



sn_veg, sn_soil = TSEB.rad.calc_Sn_Campbell(input_dict['LAI'], input_dict['SZA'], Sdn_dir, Sdn_dif, fvis, fnir,
                                                    input_dict['RHO_VIS_C'],
                                                    input_dict['TAU_VIS_C'],
                                                    input_dict['RHO_NIR_C'],
                                                    input_dict['TAU_NIR_C'],
                                                    input_dict['RHO_VIS_S'],
                                                    input_dict['RHO_NIR_S'],
                                                    x_LAD=input_dict['X_LAD'], LAI_eff=input_dict['LAI_EFF']) 

sn_veg[~np.isfinite(sn_veg)] = 0
sn_soil[~np.isfinite(sn_soil)] = 0

input_dict['SN_VEG'] = sn_veg
input_dict['SN_SOIL'] = sn_soil


print('Done!')

### Landscape roughness

In [ ]:
z_0m, d_0 = TSEB.res.calc_roughness(input_dict['LAI'],
                                    input_dict['H_C'],
                                    input_dict['W_C'],
                                    np.full_like(input_dict['LAI'], TSEB.res.CROP))
d_0[d_0 < 0] = 0
z_0m[z_0m < input_dict['Z0_SOIL']] = constant_params['Z0_SOIL']

# store in input dictionary
input_dict['d_0'] = d_0
input_dict['Z_0M'] = z_0m

print('Done!')

# Running TSEB

Now we have processed all necessary inputs to run TSEB! Let's run TSEB and store the results in an output dictionary (i.e. model_outdict)

In [ ]:
# output dictionary to save model results
model_outdict = {}

In [ ]:
# use Norman and Kustas 1995 resistance framework
Resistance_flag=[0,{}]
# using constant ratio appraoch to estimate G
G_constant = 0.35
calcG = [[1], G_constant]

[flag_PT_all, T_soil, T_veg, T_AC, Ln_soil, Ln_veg, LE_veg, H_veg,
     LE_soil, H_soil, G_mod, R_S, R_X, R_A, u_friction, L, n_iterations] = TSEB.TSEB_PT(input_dict['LST'],
                                                                                        input_dict['VZA'],
                                                                                        input_dict['T_A1'],
                                                                                        input_dict['u'],
                                                                                        input_dict['EA'],
                                                                                        input_dict['p'],
                                                                                        input_dict['SN_VEG'],
                                                                                        input_dict['SN_SOIL'],
                                                                                        input_dict['LW-IN'],
                                                                                        input_dict['LAI'],
                                                                                        input_dict['H_C'],
                                                                                        input_dict['E_V'],
                                                                                        input_dict['E_S'],
                                                                                        input_dict['Z_0M'],
                                                                                        input_dict['d_0'],
                                                                                        100,
                                                                                        100,
                                                                                        leaf_width=input_dict['LEAF_WIDTH'],
                                                                                        alpha_PT=input_dict['ALPHA_PT'],
                                                                                        f_c=input_dict['FCOVER'],
                                                                                        f_g=input_dict['F_G'],
                                                                                        calcG_params=calcG,
                                                                                        resistance_form=Resistance_flag)

# save ouputs in outdict 
LE = LE_veg + LE_soil
H = H_veg + H_soil
Rn = (Ln_veg + sn_veg) + (Ln_soil + sn_soil)

# Estimate daily ET assuming LE/Sdn ratio remains relatively constant troughout the day
le_ratio = LE/input_dict['S_dn']
ET_daily = met.flux_2_evaporation(input_dict['S_dn_24'] * le_ratio, input_dict['T_A1'], time_domain=24)

# evaporative stress index
esi = ET_daily/input_dict['ETR']

# save outputs to dictionary
model_outdict['LE_TSEB-PT'] = LE
model_outdict['LEc_TSEB-PT'] = LE_veg
model_outdict['H_TSEB-PT'] = H
model_outdict['Rn_TSEB-PT'] = Rn
model_outdict['G_TSEB-PT'] = G_mod
model_outdict['ET_TSEB-PT'] = ET_daily
model_outdict['ESI_TSEB-PT'] = esi
model_outdict['Flags_TSEB-PT'] = flag_PT_all

te = [float(s2_cube['x'].min()), float(s2_cube['x'].max()), float(s2_cube['y'].min()), float(s2_cube['y'].max())]

variables = ['LE', 'H', 'Rn', 'G']
# visualizing outputs 
#fig, axes = plt.subplots(2,2, figsize=(10,8))
i = 0
fig, axes = plt.subplots(4,2, figsize=(10,16))

for var in variables:
    # entire ROI
    ax = axes[i,0]
    if var == 'LE':
        im = ax.imshow(model_outdict[f'{var}_TSEB-PT'], vmin=0, vmax=600, cmap='PuBu', extent = te)
    elif var == 'H':
        im = ax.imshow(model_outdict[f'{var}_TSEB-PT'], vmin=0, vmax=600, cmap='OrRd',  extent = te)
    elif var == 'Rn':
        im = ax.imshow(model_outdict[f'{var}_TSEB-PT'], vmin=0, vmax=800, cmap='plasma',  extent = te)
    else:
        im = ax.imshow(model_outdict[f'{var}_TSEB-PT'], vmin=0, vmax=200, cmap='copper',  extent = te)

    ax.set_title(f'{var}', fontsize=14)
    flux_mean = int(np.round(np.nanmean(model_outdict[f'{var}_TSEB-PT']),0))
    ax.text(0.01,0.1, f'mean:\n{flux_mean} W/$m^2$', transform=ax.transAxes)
    
    # zoom to Almonds
    ax = axes[i,1]
    if var == 'LE':
        im = ax.imshow(model_outdict[f'{var}_TSEB-PT'], vmin=0, vmax=600, cmap='PuBu', extent = te)
    elif var == 'H':
        im = ax.imshow(model_outdict[f'{var}_TSEB-PT'], vmin=0, vmax=600, cmap='OrRd',  extent = te)
    elif var == 'Rn':
        im = ax.imshow(model_outdict[f'{var}_TSEB-PT'], vmin=0, vmax=800, cmap='plasma',  extent = te)
    else:
        im = ax.imshow(model_outdict[f'{var}_TSEB-PT'], vmin=0, vmax=200, cmap='copper',  extent = te)

    # Add colorbar 
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label(f'{var} (W/$m^2$)')  # Add title to colorbar
    ax.set_xlim(654900, 655850)
    ax.set_ylim(4156600, 4157500)
    ax.set_title(f'Zoom to Almond Orchard', fontsize=12)


    i = i + 1

plt.show()

:::{tip} Question
What do you think is driving the spatial patterns of modelled LE/H from TSEB? Have a look at the biophysical variables and parameters above.  
:::

# Running 3SEB

As discussed previously, 3SEB uses esentially the same inputs as TSEB but needs to parameterize both vegetation layers i.e. tree crop and cover crop.

:::{important}
In general, 3SEB should only be applied in agro-forestry systems with two distinct co-occurring vegetation layers such as savannas or tree crops with cover crops. Normally, at the satellite scale, we usually only apply 3SEB over pixels that are classified as savannas and using a global tree cover fraction product such as [**this**](https://land.copernicus.eu/en/products/high-resolution-layer-forests-and-tree-cover?tab=tree_cover_density). 

However, it is still a challenge to find global land cover maps that seperate tree crops with and without cover crops as the agronomic practices may change year to year. 

In other land types, such as grasslands, herbaceous crops, or tree crops without cover crops, it is recommend to implement TSEB.

:::

### Characterizing 3SEB specific variables

Here we will characterterize the two vegetation layers by making certain assumptions while attempting to best characterize the target Almond orchards. 

Separating the contribution of tree and grass layers within a mixed is often challenging at global scales. In orchards with cover crops, we could use information of the different phenology of tree and cover crops to seperate both signals as done in the application of 3SEB using proximal sensing (see ./303-py3SEB-Proximal). However, ideally this seperation could be done more dynamically without prescribe information, through, for example, spectral unmixing algorithms, but this is still in development.

In [ ]:
# assumed cover crop fraction cover is 1 (this refers to fraction of vegetation cover in the substate(soil+cover crop)
input_dict['Fc_C'] = np.full_like(input_dict['LAI'], 0.2)
input_dict['Fc_CC'] = np.full_like(input_dict['LAI'], 1)

# assumed tree crop has a fixed local LAI
## tree crop 
F = 2.5
# tree crop LAI
input_dict['LAI_C']=  input_dict['Fc_C'] * F

## cover crop
input_dict['LAI_CC'] =  input_dict['LAI'] - input_dict['LAI_C']
input_dict['LAI_CC'][input_dict['LAI_CC']<0] = 0

# cover crop height (assiming the Almond orchard target)
input_dict['H_C_TREE'] = np.full_like(input_dict['LAI'], 3.)
input_dict['H_CC'] = np.full_like(input_dict['LAI'], 0.4)

# green fraction
input_dict['Fg_C'] = np.full_like(input_dict['LAI'], 0.9)
input_dict['Fg_CC'] = np.full_like(input_dict['LAI'], 1)

# Leaf width
input_dict['LEAF_WIDTH_C'] = np.full_like(input_dict['LAI'], 0.05)
input_dict['LEAF_WIDTH_CC'] = np.full_like(input_dict['LAI'], 0.01)


# calculate clumping index
## tree crop
Omega0 = TSEB.CI.calc_omega0_Kustas(input_dict['LAI_C'], input_dict['Fc_C'], x_LAD=input_dict['X_LAD'], isLAIeff=True)
Omega = TSEB.CI.calc_omega_rows(input_dict['LAI_C'], input_dict['Fc_C'], theta=input_dict['SZA'],
                                psi=psi, w_c=input_dict['W_C'], x_lad=input_dict['X_LAD'])
# effective LAI (tree crop)
input_dict['LAI_EFF_C'] =  F * Omega

## understory vegetation
Omega0_un = TSEB.CI.calc_omega0_Kustas(input_dict['LAI_CC'], input_dict['Fc_CC'], x_LAD=input_dict['X_LAD'], isLAIeff=True)
Omega_un = TSEB.CI.calc_omega_Kustas(Omega0_un, input_dict['W_C'], w_C=input_dict['SZA'])

F_cc = input_dict['LAI_CC']/input_dict['Fc_CC']
# effective LAI (cover crop)
input_dict['LAI_EFF_CC'] =  F_cc * Omega_un

print('Done!')

### 3-Source Net Radiation Modeling
Estimate shortwave radiation transmission using an adapted Campbell 1998 model (adapted for 3 layers). See the Supplementary Information of [Burchard-Levine et al. (2022)](https://doi.org/10.1111/gcb.16002) for more details. 
- sn_ov = net shortwave radiation for overstory (tree crop)
- sn_un = net shortwave radiation for understory (cover crop)
- sn_soil =  net shortwave radiation for soil 

In [ ]:
# estimate radiation transmission using Campbell 1998 model (adapted for 3 layers)
sn_ov, sn_s, sn_un = py3seb.calc_Sn_Campbell(input_dict['LAI_C'],
                                       input_dict['LAI_CC'],
                                       input_dict['SZA'],
                                       Sdn_dir,
                                       Sdn_dif,
                                       fvis,
                                       fnir,
                                       input_dict['RHO_VIS_C'],
                                       input_dict['RHO_VIS_C'],
                                       input_dict['TAU_VIS_C'],
                                       input_dict['TAU_VIS_C'],
                                       input_dict['RHO_NIR_C'],
                                       input_dict['RHO_NIR_C'],
                                       input_dict['TAU_NIR_C'],
                                       input_dict['TAU_NIR_C'],
                                       input_dict['RHO_VIS_S'],
                                       input_dict['RHO_NIR_S'],
                                       input_dict['H_C_TREE'],
                                       0.25 * input_dict['H_C_TREE'],
                                       input_dict['W_C'],
                                       input_dict['Fc_C'],
                                       LAI_eff=input_dict['LAI_EFF_C'],
                                       LAI_eff_sub=input_dict['LAI_EFF_CC'])

sn_ov[~np.isfinite(sn_ov)] = 0
sn_s[~np.isfinite(sn_s)] = 0
sn_un[~np.isfinite(sn_un)] = 0

input_dict['SN_C'] = sn_ov
input_dict['SN_CC'] = sn_un
input_dict['SN_S'] = sn_s



print('Done!')

### Landscape roughness

In [ ]:
# calculate tree crop roughness parameters taking into account LAIeff using Raupach 1994 model
z_0M_factor, d_0_factor = py3seb.raupach_94(input_dict['LAI_EFF_C'])
d_0 = input_dict['H_C_TREE']*d_0_factor
z_0M = input_dict['H_C_TREE']*z_0M_factor

d_0[d_0 < 0] = 0
z_0M[z_0M < input_dict['Z0_SOIL']] = constant_params['Z0_SOIL']

# understory/secondary vegetation (i.e. Cover crop)
z_0m_un, d_0_un = TSEB.res.calc_roughness(input_dict['LAI_EFF_CC'],
                                          input_dict['H_CC'],
                                          input_dict['W_C'],
                                          np.full_like(input_dict['LAI'], TSEB.res.GRASS))
d_0_un[d_0_un < 0] = 0
z_0m_un[z_0m_un < input_dict['Z0_SOIL']] = constant_params['Z0_SOIL']

input_dict['d_0_C'] = d_0
input_dict['Z_0M_C'] = z_0M

input_dict['d_0_CC'] = d_0_un
input_dict['Z_0M_CC'] = z_0m_un

print('Done!')

## Running 3SEB-PT

In [ ]:
[flag_PT_all, T_S, T_C, T_C_sub, T_AC, L_n_sub, L_nC, Ln_C_sub, Ln_S, LE_C, H_C, LE_C_sub, H_C_sub,
 LE_S, H_S, G_mod, R_S, R_sub, R_X, R_A, u_friction, L, n_iterations] = py3seb.ThreeSEB_PT(input_dict['LST'],
                                                                                         input_dict['VZA'],
                                                                                         input_dict['T_A1'],
                                                                                         input_dict['u'],
                                                                                         input_dict['EA'],
                                                                                         input_dict['p'],
                                                                                         input_dict['SN_C'],
                                                                                         input_dict['SN_S'],
                                                                                         input_dict['SN_CC'],
                                                                                         input_dict['LW-IN'],
                                                                                         input_dict['LAI_C'],
                                                                                         input_dict['LAI_CC'],
                                                                                         input_dict['H_C_TREE'],
                                                                                         input_dict['H_CC'],
                                                                                         input_dict['E_V'],
                                                                                         input_dict['E_V'],#change e_v cover crop
                                                                                         input_dict['E_S'],
                                                                                         input_dict['Z_0M_C'],
                                                                                         input_dict['Z_0M_CC'],
                                                                                         input_dict['d_0_C'],
                                                                                         input_dict['d_0_CC'],
                                                                                         100,
                                                                                         100,
                                                                                         leaf_width=input_dict['LEAF_WIDTH_C'],
                                                                                         leaf_width_sub=input_dict['LEAF_WIDTH_CC'],
                                                                                         f_c=input_dict['Fc_C'],
                                                                                         f_c_sub=input_dict['Fc_CC'],
                                                                                         f_g=input_dict['Fg_C'],
                                                                                         f_g_sub=input_dict['Fg_CC'],
                                                                                         calcG_params=calcG,
                                                                                         resistance_form=Resistance_flag)
# save ouputs in outdict 
LE = LE_C + LE_C_sub + LE_S
H = H_C + H_S + H_C_sub

Rn_C = L_nC + sn_ov
Rn_C_sub = Ln_C_sub + sn_un
Rn_S = Ln_S + sn_s
Rn = Rn_C + Rn_C_sub + Rn_S

# Estimate daily ET assuming LE/Sdn ratio remains relatively constant troughout the day
le_ratio = LE/input_dict['S_dn']
ET_daily = met.flux_2_evaporation(input_dict['S_dn_24'] * le_ratio, input_dict['T_A1'], time_domain=24)

# evaporative stress index
esi = ET_daily/input_dict['ETR']

# save outputs to dictionary
model_outdict['LE_3SEB-PT'] = LE
model_outdict['LEc_3SEB-PT'] = LE_C
model_outdict['LEcc_3SEB-PT'] = LE_C_sub
model_outdict['H_3SEB-PT'] = H
model_outdict['Rn_3SEB-PT'] = Rn
model_outdict['G_3SEB-PT'] = G_mod
model_outdict['ET_3SEB-PT'] = ET_daily
model_outdict['ESI_3SEB-PT'] = esi

model_outdict['Flags_3SEB-PT'] = flag_PT_all

# visualizing outputs 
variables = ['LE', 'H', 'Rn', 'G']
i = 0
fig, axes = plt.subplots(4,2, figsize=(10,16))

for var in variables:
    # entire ROI
    ax = axes[i,0]
    if var == 'LE':
        im = ax.imshow(model_outdict[f'{var}_3SEB-PT'], vmin=0, vmax=600, cmap='PuBu', extent = te)
    elif var == 'H':
        im = ax.imshow(model_outdict[f'{var}_3SEB-PT'], vmin=0, vmax=600, cmap='OrRd',  extent = te)
    elif var == 'Rn':
        im = ax.imshow(model_outdict[f'{var}_3SEB-PT'], vmin=0, vmax=800, cmap='plasma',  extent = te)
    else:
        im = ax.imshow(model_outdict[f'{var}_3SEB-PT'], vmin=0, vmax=200, cmap='copper',  extent = te)

    ax.set_title(f'{var}', fontsize=14)
    flux_mean = int(np.round(np.nanmean(model_outdict[f'{var}_3SEB-PT']),0))
    ax.text(0.01,0.1, f'mean:\n{flux_mean} W/$m^2$', transform=ax.transAxes)
    
    # zoom to Almonds
    ax = axes[i,1]
    if var == 'LE':
        im = ax.imshow(model_outdict[f'{var}_3SEB-PT'], vmin=0, vmax=600, cmap='PuBu', extent = te)
    elif var == 'H':
        im = ax.imshow(model_outdict[f'{var}_3SEB-PT'], vmin=0, vmax=600, cmap='OrRd',  extent = te)
    elif var == 'Rn':
        im = ax.imshow(model_outdict[f'{var}_3SEB-PT'], vmin=0, vmax=800, cmap='plasma',  extent = te)
    else:
        im = ax.imshow(model_outdict[f'{var}_3SEB-PT'], vmin=0, vmax=200, cmap='copper',  extent = te)

    # Add colorbar 
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label(f'{var} (W/$m^2$)')  # Add title to colorbar
    ax.set_xlim(654900, 655850)
    ax.set_ylim(4156600, 4157500)
    ax.set_title(f'Zoom to Almond Orchard', fontsize=12)


    i = i + 1

plt.show()

:::{note}

In this case, the comparison between TSEB-PT and 3SEB-PT is not entirely fair since we used tailored inputs specific for the Almond orchards in 3SEB-PT for the entire scene while TSEB-PT used a dynamic vegetation parameterization based on global methods. 

You can re-run TSEB-PT using parameter values expected for the Almond Orchard and see if the results change significantly over the experimental area.
:::

:::{tip} Next steps and Questions
* You can evaluate both TSEB-PT and 3SEB-PT over the tower footprint. **Hint:** see *./403-UAV_3SEB.ipynb*
* How to these satellite-based outputs compare to the UAV results from *./403-UAV_3SEB.ipynb*?
* Any other global datasets that can be used to improve this workflow for tree crops?
:::


# Conclusions

* The Copernicus program offers a large set of products that are suitable to model ET globally
* 3SEB presents the advantage of partitioning fluxes from two distict coexisting canopies, suitable to better simluate orchards with cover crops
* However, it remains a challenge to well characterize and parameterize the two vegetation canopies solely relying on global datasets
* [...]

:::{note}
Please feel free comment any thoughts. This is work in progress!!!
:::